In [ ]:
import matplotlib.pyplot as plt
import DMK_go_coude as Fns
import numpy as np
import os, readcol
import scipy.optimize as optim
import scipy.interpolate as interp
import pickle

from astropy.io import fits 
from mpfit import mpfit
from scipy import signal

%matplotlib inline

In [ ]:
dir = os.getenv("HOME") + '/Research/YMG/coude_data/20140321/'
rdir = dir + 'reduction/'
codedir = os.getenv("HOME") + '/codes/coudereduction/'
#codedir = os.getenv("HOME") + '/Research/Codes/coudereduction/'

In [ ]:
os.chdir(dir)

In [ ]:
DarkCurVal = 0.0

InfoFile = 'headstrip.csv'
FileInfo = readcol.readcol( InfoFile, fsep = ',', asRecArray = True )
DarkCube = FileInfo.ExpTime * DarkCurVal

In [ ]:
BiasInds = np.where( FileInfo.Type == 'zero' )[0]
FlatInds = np.where( FileInfo.Type == 'flat' )[0]
ArcInds  = np.where( (FileInfo.Type == 'comp') & ( (FileInfo.Object == 'Thar') | (FileInfo.Object == 'THAR') | (FileInfo.Object == 'A') ) )[0]
ObjInds  = np.where( (FileInfo.Type == 'object') & (FileInfo.Object != 'SolPort') & (FileInfo.Object != 'solar port') & (FileInfo.Object != 'solar_ort') )[0]

In [ ]:
CalsDone = True
SuperBias, FlatField = Fns.Basic_Cals( FileInfo.File[BiasInds], FileInfo.File[FlatInds], CalsDone, rdir, plots = False )

In [ ]:
ShowBPM = False
BPM = Fns.Make_BPM( SuperBias, FlatField, 99.9, ShowBPM )

In [ ]:
RdNoise  = FileInfo.rdn[ArcInds] / FileInfo.gain[ArcInds]
DarkCur  = DarkCube[ArcInds] / FileInfo.gain[ArcInds]
ArcCube, ArcSNR = Fns.Make_Cube( FileInfo.File[ArcInds], RdNoise, DarkCur, Bias = SuperBias )

RdNoise  = FileInfo.rdn[ObjInds] / FileInfo.gain[ObjInds]
DarkCur  = DarkCube[ObjInds] / FileInfo.gain[ObjInds]
ObjCube, ObjSNR = Fns.Make_Cube( FileInfo.File[ObjInds], RdNoise, DarkCur, Bias = SuperBias, Flat = FlatField, BPM = BPM )

In [ ]:
OrderStart = -32
TraceDone = True
MedCut = 95.0
MedTrace, FitTrace = Fns.Get_Trace( FlatField, ObjCube, OrderStart, MedCut, rdir, TraceDone, plots = False )

In [ ]:
#fullspec, fullsig_spec = Fns.extractor( ObjCube, ObjSNR, FitTrace, quick = False, nosub = False, arc = False )

In [ ]:
spec     = pickle.load(open(rdir+'extracted_spec_oldway.pkl','rb'))
sig_spec = pickle.load(open(rdir+'extracted_sigspec_oldway.pkl','rb'))
wspec     = pickle.load(open(rdir+'extracted_wspec_oldway.pkl','rb'))
sig_wspec = pickle.load(open(rdir+'extracted_sigwspec_oldway.pkl','rb'))

In [ ]:
wspec      = wspec[:,::-1,:]
sig_wspec  = sig_wspec[:,::-1,:]
spec       = spec[:,::-1,:]
sig_spec   = sig_spec[:,::-1,:]

In [ ]:
def Gaussian( x, A, mean, sigma, const ):
    gauss = A * np.exp( - ( x - mean ) ** 2.0 / ( 2.0 * sigma ** 2 ) ) + const
    return gauss
    
def TwoGaussian( x, A1, mean1, sigma1, const, A2, mean2, sigma2 ):
    return Gaussian( x, A1, mean1, sigma1, const ) + Gaussian( x, A2, mean2, sigma2, const )

In [ ]:
THAR      = fits.open( codedir + 'thar_photron.fits' )[0]
THARhead  = THAR.header
THARspec  = THAR.data
THARwav   = np.arange( len( THARspec ) ) * THARhead['CDELT1'] + THARhead['CRVAL1']
THARlines = readcol.readcol( codedir + 'ThAr_list.txt', asRecArray = True )

logTHARspec = np.log10(THARspec)

#qwe            = np.where((THARwav >= np.min(roughwav-10)) & (THARwav <= np.max(roughwav+10)))[0] 
#ls             = THARspec[qwe] - np.min(THARspec[qwe])
#region         = np.where(ls < np.median(ls))
#ls[region]     = np.median(ls)

In [ ]:
roughdisp = 7000.0 / 60000.0 / 2.0
wavestart = 6957.0
startord  = wspec.shape[1] - 15 - 1
arcspec   = wspec[0,startord,:]
roughwav  = wavestart + roughdisp * np.arange( len( arcspec ) )

oldwsol     = pickle.load( open( codedir + 'global_wsol_arizz.pkl', 'rb' ) )[0]
oldstartord = oldwsol.shape[0] - 15 - 1
orderdiff   = startord - oldstartord

testroughwav = oldwsol[oldstartord]

arcspec           = arcspec - np.min( arcspec )
belowmed          = np.where( arcspec < np.median( arcspec ) )
arcspec[belowmed] = np.median( arcspec )
logarcspec        = np.log10( arcspec )
logarcspec        = logarcspec - np.min( logarcspec )

In [ ]:
def Find_Peaks_Wavelet( wav, spec, peaksnr = 5, pwidth = 10, minsep = 1 ):
    
    # Find peaks using the cwt routine from scipy.signal
    peaks = signal.find_peaks_cwt( spec, np.arange( 1, 2 ), min_snr = peaksnr, noise_perc = 20 )
    
    # Offset from start/end of spectrum by some number of pixels
    peaks = peaks[ (peaks > pwidth) & (peaks < len(spec) - pwidth) ]
    
    pixcent = np.array([])
    wavcent = np.array([])
        
    for peak in peaks:
        
        xi   = wav[peak - pwidth:peak + pwidth]
        yi   = spec[peak - pwidth:peak + pwidth]
        inds = np.arange( len(xi), dtype = float )
        
        pguess   = [ yi[9], np.median( inds ), 0.9, np.median( spec ) ]
        lowerbds = [ 0.1*pguess[0], pguess[1] - 2.0, 0.3, 0.0  ]
        upperbds = [ np.inf, pguess[1] + 2.0, 1.5, np.inf ]
        
        try:
            params, pcov = optim.curve_fit( Gaussian, inds, yi, p0 = pguess, bounds = (lowerbds,upperbds) )
            
            pixval  = peak - pwidth + params[1]
            pixcent = np.append( pixcent, pixval )
            
            ceiling = np.ceil( pixval ).astype(int)
            floor   = np.floor( pixval ).astype(int)
            slope   = ( wav[ceiling] - wav[floor] ) / ( ceiling - floor )
            wavval  = wav[floor] + slope * ( pixval - floor )
            wavcent = np.append( wavcent, wavval )
            
        except RuntimeError:
            pixval  = 'nan'
            
    vals = spec[pixcent.astype(int)]
    oks  = np.ones( len(pixcent), int )
    
    for i in range( len(wavcent) ):
        dist  = np.absolute( wavcent - wavcent[i] )
        close = np.where( dist <= minsep )[0]
        small = np.where( vals[close] < np.max( vals[close] ) )[0]
        if len(small) != 0: oks[close[small]] = -1
    keep    = np.where( oks == 1 )
    pixcent = pixcent[keep]
    wavcent = wavcent[keep]
            
    return pixcent, wavcent

def Get_WavSol( wav, spec, snrarr, pwidth = 10, minsep = 1 ):
    
    wavsols    = np.zeros( ( len(snrarr) + 1, len(spec) ) )
    wavsols[0] = wav
    wavsolsspls = np.copy( wavsols )
    
    for i in range( len(snrarr) ):
        
        pixcent, wavcent = Find_Peaks_Wavelet( wavsols[i], spec, peaksnr = snrarr[i], minsep = minsep )
        
        THARthisord = [ line for line in THARlines.wav if line > wavsols[i,0] and line < wavsols[i,-1] ]
        
        matchedpix   = np.array([])
        matchedwav   = np.array([])
        matchedlines = np.array([])
        
        for j in range( len(wavcent) ):
            dists    = np.absolute( THARthisord - wavcent[j] )
            mindist  = np.min( dists )
            mindisti = np.argmin( dists )
            
            if mindist <= 1.5:
                matchedpix   = np.append( matchedpix, pixcent[j] )
                matchedwav   = np.append( matchedwav, wavcent[j] )
                matchedlines = np.append( matchedlines, THARthisord[mindisti] )

        wavspl = interp.InterpolatedUnivariateSpline( matchedpix, matchedlines, k = 3 )
        wavsolspl = wavspl( np.arange( len(spec) ) )
        
        wavparams = np.polyfit( matchedpix, matchedlines, 4 )
        wavsol    = np.polyval( wavparams, np.arange( len(spec) ) )
        fromfit = np.polyval( wavparams, matchedpix )

        wavsols[i+1] = wavsol
        wavsolsspls[i+1] = wavsolspl
        
    return wavsols, wavsolsspls, matchedwav, matchedpix, matchedlines, fromfit

In [ ]:
wavsols, wavsolsspls, matchedwav, matchedpix, matchedlines, fromfit = Get_WavSol( testroughwav, arcspec, [100], minsep = 1.5 )
#print np.polyfit( matchedpix, matchedwav, 4 )
#print np.polyfit( matchedpix, matchedlines, 4 )

print matchedpix
print oldpeaks[:,0]
print fromfit

absdev = np.absolute( fromfit - matchedlines )
print np.median(absdev) / matchedlines * 3e5
print absdev / matchedlines * 3e5

plt.plot( matchedlines, fromfit - matchedlines, 'kx' )
plt.show()

In [ ]:
start = wavsols[-1,0]
while start<=wavsols[-1,-1]:
    plt.clf()
    plt.plot( wavsols[-1], logarcspec, 'k-' )
    plt.plot( wavsolsspls[-1], logarcspec, 'b--' )
    plt.plot( np.polyval(newfit,np.arange(len(arcspec))), logarcspec, 'g-' )
#    plt.plot( oldwav, logarcspec, 'g-' )
    plt.plot( testroughwav, logarcspec, 'm-' )
    plt.plot( THARwav, logTHARspec, 'r-' )
    for peak in matchedwav:
        plt.axvline( x = peak, color = 'm', ls=':' )
#     for peak in matchedpix:
#         plt.axvline( x = wavsols[-1,np.round(peak).astype(int)], color = 'm', ls = ':' )
    plt.xlim( start, start + 10.0 )
    plt.show()
    start += 10

In [ ]:
plt.clf()
plt.plot( wavsols[-1], logarcspec, 'k-' )
plt.plot( wavsolsspls[-1], logarcspec, 'b--' )
plt.plot( oldwav, logarcspec, 'g-' )
plt.plot( testroughwav, logarcspec, 'm:' )
plt.plot( THARwav, logTHARspec, 'r-' )
plt.xlim( 6965, 6968 )
plt.show()
print THARthisord

In [ ]:
# thisord = np.where( ( THARwav >= roughwav[0] - 5 ) & ( THARwav <= roughwav[-1] + 5 ) )
# thisTHARwav  = THARwav[thisord]
# thisTHARspec = THARspec[thisord]
# print thisTHARwav
# THARpixcent, THARwavcent = Find_Peaks_Wavelet( thisTHARwav, thisTHARspec, peaksnr=3 )

# THARthisord = [ line for line in THARlines.wav if line > thisTHARwav[0] and line < 7100 ]

# plt.clf()
# plt.plot( thisTHARwav, np.log10(thisTHARspec), 'k-' )
# for peak in THARwavcent:
#     plt.axvline( x = peak, color = 'r', ls = ':' )
# plt.xlim( 7050, 7080 )
# plt.show()

In [ ]:
print oldpeaks
testpix = [ 130.50867382, 541.78811423, 734.18960052, 1045.23517069, 1252.9122021, 1927.71519102, 1956.05814789 ]
testwav = [ 6965.43089741, 6989.65599411, 7000.80324912, 7018.5681611, 7030.25082966, 7067.21651247, 7068.73592592 ]
print THARthisord
newfit = np.polyfit( testpix, oldpeaks[:,1], 4 )

In [ ]:
oldpeaks = pickle.load( open( '/Users/dmk2347/Desktop/ARizzRed/peaks_ord_32.pkl', 'rb' ) )
oldparams = pickle.load( open( '/Users/dmk2347/Desktop/ARizzRed/params_wsol.pkl', 'rb' ) )
oldfit   = np.polyfit( oldpeaks[:,0], oldpeaks[:,1], 4 )
oldwav = np.polyval( oldfit, np.arange(len(arcspec) ) )

plt.plot( oldwav, logarcspec, 'k-' )
plt.plot( THARwav, logTHARspec, 'r-' )
plt.xlim( 6955, 6980 )
plt.show()

In [ ]:
arcspec           = wspec[0,startord,:]
arcspec           = arcspec - np.min( arcspec )
belowmed          = np.where( arcspec < np.median( arcspec ) )
arcspec[belowmed] = np.median( arcspec )
logarcspec        = np.log10( arcspec )
logarcspec        = logarcspec - np.min( logarcspec )

testwav = np.polyval( oldfit, np.arange(len(arcspec)) )
plt.plot( testwav, logarcspec, 'k-' )
plt.plot( THARwav, logTHARspec, 'r-' )
for peak in oldpeaks:
    plt.axvline( x = peak[1], color = 'b', )
plt.xlim( 6960, 6970 )
plt.show()